In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, lit, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import json

In [ ]:
spark = SparkSession.builder.appName("SocialMediaBotDetection").getOrCreate()

In [ ]:
df_raw = spark.read.json("/content/TwiBot-20_sample.json")
df_raw.printSchema()


In [ ]:
df_raw.select('ID', 'domain', 'profile', 'tweet', 'neighbor').show(5)

In [ ]:
df = df_raw.fillna("")

In [ ]:
from pyspark.sql.functions import concat_ws, col

df = df.withColumn(
    "combined_text",
    concat_ws(" ",
              col("profile.description"),
              col("profile.name"),
              concat_ws(" ", col("tweet"))  # flatten the array of tweet strings
             )
)

In [ ]:
df.select("profile.*").printSchema()

In [ ]:
df = df.withColumn("followers_count_int", col("profile.followers_count").cast("int"))

In [ ]:
df = df.withColumn(
    "label",
    when(col("profile.description").contains("bot") | (col("followers_count_int") < 50), 1).otherwise(0)
)

In [ ]:
tokenizer = Tokenizer(inputCol="combined_text", outputCol="words")
words_data = tokenizer.transform(df)

In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_data = remover.transform(words_data)

In [ ]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=5000)
featurized_data = hashingTF.transform(filtered_data)

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)

In [ ]:
final_data = rescaled_data.select("features", "label")

In [ ]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)

In [ ]:
predictions = lr_model.transform(test_data)
predictions.select("label", "prediction", "probability").show(5)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, when, length
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialize Spark Session
spark = SparkSession.builder.appName("AdvancedBotDetection").getOrCreate()

# Load Data
df = spark.read.json("/content/TwiBot-20_sample.json")
df = df.fillna("")

In [ ]:
from pyspark.sql.functions import concat_ws, col

df = df.withColumn(
    "combined_text",
    concat_ws(" ",
              col("profile.description"),
              col("profile.name"),
              concat_ws(" ", col("tweet"))  # flatten the array of tweet strings
             )
)

In [ ]:
df = df.withColumn("followers_count_int", col("profile.followers_count").cast("int"))
df = df.withColumn(
    "label",
    when(col("profile.description").contains("bot") | (col("followers_count_int") < 50), 1).otherwise(0)
)

In [ ]:
df.select("label").groupBy("label").count().show()
df = df.withColumn("text_length", length("combined_text"))

In [ ]:
tokenizer = Tokenizer(inputCol="combined_text", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

In [ ]:
word2vec = Word2Vec(vectorSize=100, minCount=2, inputCol="filtered_tokens", outputCol="features")

In [ ]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
gbt = GBTClassifier(featuresCol="features", labelCol="label")

In [ ]:
pipeline = Pipeline(stages=[tokenizer, remover, word2vec, lr])

In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
paramGrid = ParamGridBuilder().addGrid(word2vec.vectorSize, [50, 100]).addGrid(lr.regParam, [0.01, 0.1]).build()

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [ ]:
cv_model = crossval.fit(train)
predictions = cv_model.transform(test)

In [ ]:
accuracy = evaluator.evaluate(predictions)
print(f"Optimized Model F1 Score: {accuracy:.4f}")
predictions.select("label", "prediction").groupBy("label", "prediction").count().show()

In [ ]:
def evaluate_model(classifier, name):
    pipe = Pipeline(stages=[tokenizer, remover, word2vec, classifier])
    model = pipe.fit(train)
    pred = model.transform(test)
    score = evaluator.evaluate(pred)
    print(f"{name} F1 Score: {score:.4f}")

In [ ]:
evaluate_model(rf, "Random Forest")
evaluate_model(gbt, "Gradient Boosted Trees")

In [ ]:
cv_model.bestModel.write().overwrite().save("best_bot_detector_model")

In [ ]:
model = PipelineModel.load("best_bot_detector_model")

In [ ]:
!zip -r best_bot_detector_model.zip /content/best_bot_detector_model


In [45]:
from google.colab import files
files.download("best_bot_detector_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>